# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
# load and read saved csv
csv_file = os.path.join('..', 'WeatherPy', 'output_data', 'city_weather.csv')
city_df = pd.read_csv(csv_file)
city_df.dropna()
city_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,6165406,Thompson,CA,55.7435,-97.8558,23.16,93,90,4.61
1,4030556,Rikitea,PF,-23.1203,-134.9692,72.95,74,99,11.50
2,5882953,Aklavik,CA,68.2191,-135.0107,17.60,93,90,12.66
3,6201424,Mataura,NZ,-46.1927,168.8643,46.96,100,100,1.21
4,1507390,Dikson,RU,73.5069,80.5464,-2.76,91,99,9.84
...,...,...,...,...,...,...,...,...,...
566,1789065,Ankang,CN,32.6800,109.0172,46.20,67,0,0.94
567,102527,Sakakah,SA,29.9697,40.2064,69.37,21,100,15.28
568,64814,Bandarbeyla,SO,9.4942,50.8122,79.02,71,25,16.67
569,1166652,Saddiqabad,PK,28.3062,70.1307,71.56,16,0,5.95


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# configure gmaps api key
gmaps.configure(api_key=g_key)

In [15]:
# store lat and lng into locations
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]
#plot heatmap
fig = gmaps.figure(center = [0,0], zoom_level = 2)
#create map overlayed with heatmap
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 4
fig.add_layer(heat_layer)
# display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# create ideal dataframe with set conditions
ideal_df = city_df[(city_df['Temperature'] >= 70) & (city_df['Temperature'] <= 80)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Clouds'] <= 0]
hotel_df = ideal_df
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
68,4726521,San Patricio,US,28.0170,-97.5169,70.21,24,0,9.95
183,1244926,Hambantota,LK,6.1241,81.1185,79.14,89,0,5.75
233,347634,Ţahţā,EG,26.7693,31.5021,74.41,36,0,4.50
292,1163054,Turbat,PK,26.0023,63.0440,75.15,15,0,2.71
476,4003938,Jiménez,MX,28.3333,-105.4000,74.14,26,0,5.91
485,286621,Salalah,OM,17.0151,54.0924,78.89,57,0,3.44
569,1166652,Saddiqabad,PK,28.3062,70.1307,71.56,16,0,5.95


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# add hotel name into dataframe
hotel_df['Hotel Name'] = ""
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
68,4726521,San Patricio,US,28.0170,-97.5169,70.21,24,0,9.95,
183,1244926,Hambantota,LK,6.1241,81.1185,79.14,89,0,5.75,
233,347634,Ţahţā,EG,26.7693,31.5021,74.41,36,0,4.50,
292,1163054,Turbat,PK,26.0023,63.0440,75.15,15,0,2.71,
476,4003938,Jiménez,MX,28.3333,-105.4000,74.14,26,0,5.91,
485,286621,Salalah,OM,17.0151,54.0924,78.89,57,0,3.44,
569,1166652,Saddiqabad,PK,28.3062,70.1307,71.56,16,0,5.95,


In [22]:
# loop through each ideal location and find nearest hotel within 5000 radius added excpetion handling
for index, row in hotel_df.iterrows():
    try:
        # setting up url and payload to get required data
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"
        
        # create dataframe for information
        hotel_data = requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = "No Information on Hotel"
        
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
68,4726521,San Patricio,US,28.0170,-97.5169,70.21,24,0,9.95,"Motel 6 Sinton, TX"
183,1244926,Hambantota,LK,6.1241,81.1185,79.14,89,0,5.75,The Peacock Beach Hotel Hambantota
233,347634,Ţahţā,EG,26.7693,31.5021,74.41,36,0,4.50,منتجع ال فؤاد عباس
292,1163054,Turbat,PK,26.0023,63.0440,75.15,15,0,2.71,Khand a Hotel
476,4003938,Jiménez,MX,28.3333,-105.4000,74.14,26,0,5.91,No Information on Hotel
485,286621,Salalah,OM,17.0151,54.0924,78.89,57,0,3.44,Muscat International Hotel plaza Salalah
569,1166652,Saddiqabad,PK,28.3062,70.1307,71.56,16,0,5.95,MIDWAY HOTEL & RESTAURANT


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))